In [1]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [2]:
conf = Config()

conf.dataset = 'usa_large_cell_512'
conf.post_value_updates()
# conf.cell_embedding_dim = 256
# conf.seq_embedding_dim = 256
# conf.moco_proj_dim = conf.seq_embedding_dim//2


In [3]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
model

TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBTimeWeighted(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): 

In [5]:
# from ah_databricks_data_loader import DatabricksDataLoader

# # Instantiate the DatabricksDataLoader.
# ddl = DatabricksDataLoader()

# # Load data.
# test_df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_traj_data_v18")

In [6]:
# test_df.count()

In [7]:
# test_df.show(5)

In [8]:
# test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18")

In [9]:
# test_df.count()

In [10]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/nyc/test/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

Total records in repartitioned files: 1445257


In [11]:
df_list[0].head()

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
0,34299,2024-12-26,"[2024-12-26T00:07:27.000000000, 2024-12-26T00:...","[[-74.0597008, 40.7563587], [-74.0598405, 40.7...",u s postal service,0,5070.59,train,57,"[[-8244288.181358196, 4976470.777401652], [-82...",3,"[0, 2, 4, 5, 6, 7, 8, 10, 12, 14, 15, 16, 18, ..."
1,35404,2024-10-13,"[2024-10-13T02:49:10.000000000, 2024-10-13T03:...","[[-73.9455783, 40.7803021], [-73.9452042, 40.7...",Con edison,0,140.84,train,46,"[[-8231584.122770139, 4979990.082396612], [-82...",6,"[16, 22, 35, 39, 41, 43, 45, 48, 49, 50, 54, 5..."
2,54320,2025-05-19,"[2025-05-19T01:12:10.000000000, 2025-05-19T01:...","[[-73.9089767, 40.8332557], [-73.905857, 40.83...",Sterling Nat Bnk,0,0.00,train,33,"[[-8227509.651295921, 4987777.940954166], [-82...",0,"[7, 8, 19, 20, 24, 27, 31, 49, 52, 58, 60, 67,..."
3,54320,2025-06-10,"[2025-06-10T00:00:59.000000000, 2025-06-10T00:...","[[-73.9059267, 40.8341983], [-73.9059307, 40.8...",Sterling Nat Bnk,0,0.00,train,27,"[[-8227170.126849, 4987916.624968072], [-82271...",1,"[0, 4, 20, 27, 42, 44, 45, 47, 53, 63, 65, 70,..."
4,74093,2025-03-08,"[2025-03-08T02:09:46.000000000, 2025-03-08T02:...","[[-74.1093717454287, 40.813244574627454], [-74...",T-Mobile USA,0,1603.39,train,27,"[[-8249817.525710539, 4984834.184550401], [-82...",5,"[12, 14, 15, 16, 17, 18, 53, 54, 55, 56, 57, 1..."


In [12]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [13]:
# userids = test_df['userid'].unique()

In [14]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/usa_neg_sampling_smallqueue/ep1_batch100000"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_1557/3021168539.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBTimeWeighted(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): 

In [15]:
from utils.traj import *
import pickle
from utils.cellspace import *

from torch.nn.utils.rnn import pad_sequence
embs_parent = pickle.load(open(Config.dataset_embs_file_parent, 'rb')).to('cpu').detach() # tensor
embs_child = pickle.load(open(Config.dataset_embs_file_child, 'rb')).to('cpu').detach() # tensor
cellspace_parent = pickle.load(open(Config.dataset_cell_file_parent, 'rb'))
cellspace_child = pickle.load(open(Config.dataset_cell_file_child, 'rb'))
hier_cellspace = HirearchicalCellSpace(cellspace_parent, cellspace_child)

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell_parent, traj_cell_child, traj_p, traj_timedelta = zip(*[merc2cell2(l,t, hier_cellspace) for l,t in zip(traj, time_indices)])
    # print(traj_cell)
    traj_emb_p = [torch.tensor(generate_spatial_features(t, hier_cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell_parent = [embs_parent[list(t)] for t in traj_cell_parent]
    traj_emb_cell_child = [embs_child[list(t)] for t in traj_cell_child]
    traj_emb_cell = [a + b for a, b in zip(traj_emb_cell_parent, traj_emb_cell_child)]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_p)), dtype = torch.long, device = device)
    traj_timedelta = pad_sequence([torch.log(torch.tensor(t)) for t in traj_timedelta], batch_first=False, padding_value=0).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret_v2(traj_emb_cell.float(), traj_emb_p.float(), traj_len, traj_timedelta)
    return traj_embs, traj_cell_parent, traj_cell_child , traj_p, traj_timedelta

def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size] 
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [16]:
from sklearn.metrics.pairwise import cosine_similarity


In [17]:
df_list[0]['time_index_list'].values[0]

array([  0,   2,   4,   5,   6,   7,   8,  10,  12,  14,  15,  16,  18,
        19,  22,  26,  27,  28,  29,  31,  32,  34,  35,  36,  39,  40,
        41,  43,  45,  50,  51,  53,  55,  56,  57,  75,  76,  77, 109,
       113, 117, 122, 124, 125, 126, 128, 129, 130, 131, 133, 134, 136,
       137, 139, 140, 142, 143], dtype=int32)

In [18]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

Total unique userids: 25831
Total unique employernames: 13911


In [19]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['timestamps'].values
    return traj, time_indices

In [20]:
# def get_min_max_xy(traj):
#     min_x = min([p[0] for p in traj])
#     max_x = max([p[0] for p in traj])
#     min_y = min([p[1] for p in traj])
#     max_y = max([p[1] for p in traj])
#     return min_x, max_x, min_y, max_y

# def calculate_dx_dy(k, target_min_x, target_min_y, min_x, max_x, min_y, max_y):
#     dx = target_min_x+ (k - max_x - min_x) / 2
#     dy = target_min_y+ (k - max_y - min_y) / 2
#     return dx, dy

# def transform_traj(traj, dx, dy):
#     new_traj = [[p[0] + dx, p[1] + dy] for p in traj]
#     return new_traj

# def normalize(trajs, cellspace):
#     # trajs: list of [[lon, lat], [,], ...]

#     # 1. augment the input traj in order to form 2 augmented traj views
#     # 2. convert augmented trajs to the trajs based on mercator space by cells
#     # 3. read cell embeddings and form batch tensors (sort, pad)

#     trajs1, trajs2 = [], []
#     time_indices1, time_indices2 = [], []
#     final_min_x = 1000000000
#     final_min_y = 1000000000
#     final_max_x = -1000000000
#     final_max_y = -1000000000
#     max_len_meters = min(cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min)
#     indices_to_omit = []
#     for i, l in enumerate(trajs):
#         min_x, max_x, min_y, max_y = get_min_max_xy(l)
#         # if final_max_x - min_x> max_len_meters or final_max_y-min_y> max_len_meters or max_x - final_min_x> max_len_meters or max_y - final_min_y> max_len_meters or max_x-min_x> max_len_meters or max_y-min_y> max_len_meters:
#         #     indices_to_omit.append(i)
#         #     continue
#         final_min_x = min(final_min_x, min_x)
#         final_min_y = min(final_min_y, min_y)
#         final_max_x = max(final_max_x, max_x)
#         final_max_y = max(final_max_y, max_y)
    
#     if max(final_max_x-final_min_x, final_max_y-final_min_y)>max_len_meters:
#         # print(max(final_max_x-final_min_x, final_max_y-final_min_y), max_len_meters)
#         return None
#     dx, dy = calculate_dx_dy(max_len_meters, cellspace.x_min, cellspace.y_min, final_min_x, final_max_x, final_min_y, final_max_y)
#     # print(final_min_x, final_max_x, final_min_y, final_max_y)
#     # print(dx, dy)
#     for i,l in enumerate(trajs):
#         if i in indices_to_omit:
#             continue
#         new_l = transform_traj(l, dx, dy)
#         trajs1.append(new_l)
#     return trajs1

In [33]:

def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [34]:
from tqdm import tqdm
import torch.nn as nn
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    count = 0
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                continue
            else:
                # print(userid, employer, partition)
                # print(len(part_data))
                traj, time_indices = get_traj_and_time_data(part_data)
                # print(traj)
                embs, traj_cell_parent, traj_cell_child , traj_p, traj_timedelta = infer(traj, time_indices)
                embs = nn.functional.normalize(embs, dim=1)
                db_scan = apply_dbscan(embs.detach().cpu().numpy(), target_min_similarity=0.90)
                labels = db_scan.labels_                      # shape: (n_samples,)
                cluster_ids = [c for c in np.unique(labels) if c != -1]
                if len(cluster_ids)>0:
                    pred_label = 1
                else:
                    pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    return count
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label
count=0
for i, userid in tqdm(enumerate(unique_userids)):
    count+=get_gt_and_pred_label(userid)
    if i>1000:
        break

1001it [01:15, 13.21it/s]


In [33]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/usa_neg_sampling_smallqueue/nyc_test_hyp1_output_dict.pkl", 'wb'))

In [3]:
import pickle
output_dict = pickle.load(open("/home/sagemaker-user/TrajCL/exp/usa_neg_sampling_smallqueue/nyc_test_hyp1_output_dict.pkl","rb"))

In [35]:
output_dict

{(34299, 'u s postal service', 0): 1,
 (34299, 'u s postal service', 1): 1,
 (35404, 'Con edison', 0): 1,
 (54320, 'Sterling Nat Bnk', 0): 1,
 (74093, 'T-Mobile USA', 0): 0,
 (92918, 'Barker Consulting', 0): 1,
 (104834, 'NYC Department of Finance', 1): 1,
 (104834, 'NYC Department of Finance', 0): 1,
 (104834, 'city of new york', 0): 1,
 (131568, 'Argo realty', 1): 1,
 (131568, 'Argo realty', 0): 1,
 (132352, 'CARE DESIGN NY L', 0): 1,
 (132352, 'care design ny', 0): 1,
 (132352, 'care design ny', 1): 1,
 (134256, 'TALENTBURST-OSV', 0): 1,
 (134256, 'USPS', 0): 1,
 (134256, 'USPS', 1): 0,
 (134952, 'AT&T', 0): 1,
 (134952, 'AT&T', 1): 1,
 (135444, 'Beacon Mobil-OSV', 0): 1,
 (135444, 'Beacon Mobil-OSV', 1): 1,
 (135444, 'Eventbrite', 0): 0,
 (136830, 'Advance Auto Parts', 0): 0,
 (150194, 'Nyc health and hospitals', 1): 1,
 (150194, 'Nyc health and hospitals', 0): 1,
 (151673, 'ASAK', 0): 1,
 (154878, 'Entertainment Partners', 0): 0,
 (171723, 'Revlon', 0): 0,
 (209240, 'NewYork-Presb

In [36]:
count_dict = {}
for k,v in output_dict.items():
    if v in count_dict:
        count_dict[v]+=1
    else:
        count_dict[v]=1
count_dict

{1: 1245, 0: 294}

In [36]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part_fil = df_part_fil[df_part_fil['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part_fil

In [18]:
output_dict[(937437, 'SCO Family of Services', 1)]

1

In [38]:
df_part = get_data_userid_employer_partition(2523559, 'united ground express', 0)
df_part.sort_values('traj_date')

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
12,2523559,2024-07-10,"[2024-07-10T03:13:20.000000000, 2024-07-10T03:...","[[-74.17744913276556, 40.729217529296875], [-7...",united ground express,0,162.02,train,33,"[[-8257395.865803414, 4972482.982549811], [-82...",2,"[19, 20, 24, 31, 34, 35, 36, 37, 38, 40, 43, 4..."
0,2523559,2024-07-12,"[2024-07-12T03:10:25.000000000, 2024-07-12T03:...","[[-74.17639935377976, 40.728851318359375], [-7...",united ground express,0,162.02,train,25,"[[-8257279.004941269, 4972429.187074517], [-82...",4,"[19, 20, 23, 25, 27, 28, 29, 30, 33, 34, 38, 4..."
15,2523559,2024-07-15,"[2024-07-15T03:17:50.000000000, 2024-07-15T03:...","[[-74.1774103360832, 40.72918701171875], [-74....",united ground express,0,162.02,train,24,"[[-8257391.546976489, 4972478.499582228], [-82...",0,"[19, 20, 38, 67, 75, 76, 81, 83, 84, 86, 100, ..."
20,2523559,2024-07-17,"[2024-07-17T03:16:18.000000000, 2024-07-17T03:...","[[-74.17720047375575, 40.7291259765625], [-74....",united ground express,0,162.02,train,16,"[[-8257368.18520906, 4972469.533653232], [-825...",2,"[19, 20, 33, 70, 72, 75, 76, 87, 88, 89, 90, 9..."
11,2523559,2024-08-02,"[2024-08-02T00:24:41.000000000, 2024-08-02T01:...","[[-74.1820278713443, 40.732208251953125], [-74...",united ground express,0,133.65,train,17,"[[-8257905.568650476, 4972922.323346658], [-82...",4,"[2, 7, 37, 38, 39, 41, 52, 56, 63, 66, 70, 82,..."
13,2523559,2024-08-05,"[2024-08-05T03:22:13.000000000, 2024-08-05T03:...","[[-74.17703202885093, 40.729095458984375], [-7...",united ground express,0,133.65,train,18,"[[-8257349.43400803, 4972465.050691818], [-825...",0,"[20, 21, 49, 75, 76, 84, 85, 89, 91, 92, 93, 9..."
22,2523559,2024-08-08,"[2024-08-08T02:10:30.000000000, 2024-08-08T03:...","[[-74.17668776763831, 40.728973388671875], [-7...",united ground express,0,133.65,train,27,"[[-8257311.111025142, 4972447.11886672], [-825...",3,"[13, 22, 26, 27, 28, 29, 32, 34, 35, 36, 39, 4..."
4,2523559,2024-08-09,"[2024-08-09T03:06:29.000000000, 2024-08-09T03:...","[[-74.18203516550956, 40.732208251953125], [-7...",united ground express,0,133.65,train,28,"[[-8257906.380633238, 4972922.323346658], [-82...",4,"[18, 20, 21, 22, 26, 35, 37, 43, 59, 66, 71, 7..."
19,2523559,2024-08-12,"[2024-08-12T00:18:29.000000000, 2024-08-12T03:...","[[-74.18200103546091, 40.732269287109375], [-7...",united ground express,0,133.65,train,15,"[[-8257902.581293602, 4972931.2896909835], [-8...",0,"[1, 20, 21, 23, 75, 76, 80, 81, 87, 91, 92, 96..."
21,2523559,2024-08-13,"[2024-08-13T04:22:19.000000000, 2024-08-13T04:...","[[-74.17717739895119, 40.7291259765625], [-74....",united ground express,0,133.65,train,23,"[[-8257365.616533566, 4972469.533653232], [-82...",1,"[26, 27, 33, 34, 35, 45, 69, 74, 77, 79, 82, 8..."


In [ ]:
import numpy as np

def cosine_similarity_matrix(X, eps=1e-8, dtype=np.float32):
    X = np.asarray(X, dtype=dtype)
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    Xn = X / np.maximum(norms, eps)
    return Xn @ Xn.T

In [ ]:
def get_cluster(part_df):
    traj , time_indices= get_traj_and_time_data(part_df)
    embs, _, _, _, _ = infer(traj,time_indices)
    db_scan = apply_dbscan(embs.detach().cpu().numpy(), target_min_similarity=0.85)
    labels = db_scan.labels_                      # shape: (n_samples,)
    cluster_ids = [c for c in np.unique(labels) if c != -1]
    for i in part_df.index:
        print(labels[i], part_df.iloc[i]['traj_date'])
    sim = cosine_similarity_matrix(embs.detach().cpu().numpy())
    sim_pairs = []
    for i in range(sim.shape[0]):
        for j in range(i+1, sim.shape[1]):
            if sim[i,j]>0.85:
                sim_pairs.append((i, j, sim[i,j]))
    return sim_pairs

In [ ]:
def get_similarity(part_df):
    traj_dates = list(part_df['traj_date'])
    traj , time_indices= get_traj_and_time_data(part_df)
    # traj = normalize(traj, cellspace)
    # print(traj)
    embs, parent_cell, child_cell, traj_p, time_delta = infer(traj,time_indices)
    similarity = cosine_similarity_matrix(embs.detach().cpu().numpy())


    return similarity, parent_cell, child_cell, traj_p, time_delta

In [ ]:
import datetime

In [ ]:
get_cluster(df_part)

In [ ]:
df_fil= df_part[df_part['traj_date'].isin([datetime.date(2024,9,19), datetime.date(2024,9,13)])]
sim, parent_cell, child_cell, traj_p, time_delta = get_similarity(df_fil)

In [ ]:
sim

In [ ]:
time_delta = time_delta.cpu().numpy()

In [ ]:

sorted_indices_0 = list(np.argsort(time_delta[:,0]))[::-1]

cell_parent_fil_0 = np.array(parent_cell[0])[sorted_indices_0[:10]]
cell_fil_0 = np.array(child_cell[0])[sorted_indices_0[:10]]
timestamps_0 = time_delta[sorted_indices_0[:10],0]

sorted_indices_1 = list(np.argsort(time_delta[:,1]))[::-1]
cell_parent_fil_1 = np.array(parent_cell[1])[sorted_indices_1[:10]]
cell_fil_1 = np.array(child_cell[1])[sorted_indices_1[:10]]
timestamps_1 = time_delta[sorted_indices_1[:10],1]


for i in range(len(cell_fil_0)):
    print(cell_parent_fil_0[i], cell_fil_0[i], timestamps_0[i], math.exp(timestamps_0[i]), cell_parent_fil_1[i], cell_fil_1[i], timestamps_1[i], math.exp(timestamps_1[i]))

In [ ]:
print(sorted_indices_0)
print(sorted_indices_1)